<a href="https://colab.research.google.com/github/rrfsantos/BNDES-premiodadosabertos/blob/main/Banco_Servicos_scrap_contratos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!sudo apt-get install poppler-utils
!sudo apt-get install tesseract-ocr-por
!python -m pip install pdf2image
!python -m pip install pytesseract

In [ ]:
import pandas as pd
import numpy as np
import requests, zipfile, io
import glob
import os
import json
from pdf2image import convert_from_path
import pytesseract # Módulo para a utilização da tecnologia OCR
from pytesseract import Output

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

workdir_path = '/content/drive/My Drive/BNDES/Banco de Servicos/'
os.chdir(workdir_path)

Mounted at /content/drive


In [ ]:
df1 = pd.read_csv('banco-servicos-banco-de-servicos.csv', sep=';', decimal=',', encoding='UTF-8')

In [ ]:
# Download e descompactação dos contratos do Banco de Serviços
lista_projetos = df1['link_projeto'].to_list()
for i in range(len(lista_projetos)):
  r = requests.get(lista_projetos[i])
  z = zipfile.ZipFile(io.BytesIO(r.content))
  z.extractall('contratos')

In [ ]:
# Selecionar pdfs de contratos
pdf_path = glob.glob('contratos/*')
pdf_contratos = []
for i in range(len(pdf_path)):
  if 'contrato' in (os.path.basename(pdf_path[i])).lower():
    pdf_contratos.append(pdf_path[i])

In [ ]:
# Tranformação dos pdfs de contratos em imagems
for i in range(len(pdf_contratos)):
    pages = convert_from_path(pdf_contratos[i])
    for j in range(len(pages)):
      page = pages[j]
      page.save('imagens/' + os.path.basename(pdf_contratos[i]).split('.')[0] + '_{}.jpg'.format(j), 'JPEG')

In [ ]:
# Processamento das imagens com o Tesseract
texto = dict()
df2 = pd.DataFrame()
lista_imagens = glob.glob('imagens/*')
for i in range(len(lista_imagens)):
  texto['X'+str(i)] = pytesseract.image_to_data(lista_imagens[i], lang ='por', output_type=Output.DICT)
  d = pd.DataFrame(texto['X'+str(i)])
  df2 = df2.append(d, ignore_index=True)

In [ ]:
df2.tail()

In [ ]:
df2.to_csv('contratos.csv', sep=';', encoding='UTF-8', index=False)